# Efficient Net - Repurposing/Finetuning
## Introduction

This notebook is an attempt to repurpose and finetune an EfficientNet model to the task of American Sign Language detection for the DSPRO2 project at HSLU.

## Setup
In this section all the necessary libraries are imported.

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import wandb
import torch
import torch.nn as nn
import torchvision.models as visionmodels
import torchvision.transforms as transforms
import lightning as L

from lightning.pytorch.loggers import WandbLogger

import nbformat

import os

# Our own modules
from datapipeline.asl_image_data_module import ASLImageDataModule
from models.asl_model import ASLModel
from models.training import sweep, train

In [3]:
os.environ["WANDB_NOTEBOOK_NAME"] = "./dspro2/efficientnet.ipynb"

## Preprocessing
No general data preprocessing is necessary, however there will be random transforms applied to the images during training. The images are resized to 224x224 pixels, which is the input size of the EfficientNet model. The images are also normalized using the mean and standard deviation of the ImageNet dataset, which is the dataset on which the EfficientNet model was pretrained.

The following cells will show the loading of the dataset and the preparation of the mentioned transforms.

In [4]:
PATH = "/exchange/dspro2/silent-speech/ASL_Pictures_Dataset"

In [5]:
img_size = 224

# See https://pytorch.org/vision/master/auto_examples/transforms/plot_transforms_illustrations.html#sphx-glr-auto-examples-transforms-plot-transforms-illustrations-py
# for more examples of transforms

# Open Idea: Grayscale for anti bias


data_transforms = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1), # Idea: ColorJitter for anti bias
    transforms.RandomRotation(degrees=5),
    transforms.RandomPerspective(distortion_scale=0.5, p=0.5),
    transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # ImageNet stats
])

In [6]:
datamodule = ASLImageDataModule(path=PATH, transforms=data_transforms, val_split_folder="Validation", batch_size=32, num_workers=128)

## Models

In [7]:
NUM_CLASSES = 28

In [8]:
class ASLEfficientNetRepurpose(nn.Module):
    def __init__(self, efficientnet_model: visionmodels.efficientnet.EfficientNet, dropout: float = 0.2, num_classes: int = NUM_CLASSES):
        super().__init__()
        self.model = efficientnet_model
        self.model.requires_grad_(False)
        self.model.classifier = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(self.model.classifier[1].in_features, num_classes)
        )

    def forward(self, x):
        return self.model(x)

In [9]:
class ASLEfficientNetFinetune(ASLEfficientNetRepurpose):
    def __init__(self, efficientnet_model: visionmodels.efficientnet.EfficientNet, dropout: float = 0.2, unfreeze_features: int = 1, num_classes: int = NUM_CLASSES):
        super().__init__(efficientnet_model, dropout, num_classes)

        assert unfreeze_features > 0, "unfreeze_features must be greater than 0"
        assert unfreeze_features <= len(self.model.features), "unfreeze_features must be less than or equal to the number of features in the model"

        self.model.features[-unfreeze_features:].requires_grad_(True)

## Training

In [10]:
TUNE_TYPE = "tune_type"
EFFICIENTNET_MODEL = "efficientnet_model"

DROPOUT = "dropout"

NAME = "name"

In [11]:
def get_pretrained_efficientnet_model(model_type: str):
    if model_type == "b0":
        efficientnet_model = visionmodels.efficientnet_b0(weights=visionmodels.EfficientNet_B0_Weights.DEFAULT)
    elif model_type == "b1":
        efficientnet_model = visionmodels.efficientnet_b1(weights=visionmodels.EfficientNet_B1_Weights.DEFAULT)
    elif model_type == "b2":
        efficientnet_model = visionmodels.efficientnet_b2(weights=visionmodels.EfficientNet_B2_Weights.DEFAULT)
    elif model_type == "b3":
        efficientnet_model = visionmodels.efficientnet_b3(weights=visionmodels.EfficientNet_B3_Weights.DEFAULT)
    elif model_type == "b4":
        efficientnet_model = visionmodels.efficientnet_b4(weights=visionmodels.EfficientNet_B4_Weights.DEFAULT)
    elif model_type == "b5":
        efficientnet_model = visionmodels.efficientnet_b5(weights=visionmodels.EfficientNet_B5_Weights.DEFAULT)
    elif model_type == "b6":
        efficientnet_model = visionmodels.efficientnet_b6(weights=visionmodels.EfficientNet_B6_Weights.DEFAULT)
    elif model_type == "b7":
        efficientnet_model = visionmodels.efficientnet_b7(weights=visionmodels.EfficientNet_B7_Weights.DEFAULT)

    return efficientnet_model

In [12]:
def get_asl_efficientnet_model(type: str, efficientnet_model: visionmodels.efficientnet.EfficientNet, dropout: float, unfreeze_features: int = 1) -> nn.Module:
    if type == "repurpose":
        model = ASLEfficientNetRepurpose(efficientnet_model, dropout=dropout)
    elif type == "finetune":
        model = ASLEfficientNetFinetune(efficientnet_model, dropout=dropout, unfreeze_features=unfreeze_features)
    else:
        raise ValueError(f"Invalid model type: {type}")

    return model

In [13]:
OPTIMIZER = "optimizer"
LEARNING_RATE = "learning_rate"
WEIGHT_DECAY = "weight_decay"
MOMENTUM = "momentum"


def get_optimizer(optimizer_params: dict, model: nn.Module):
    optimizer = optimizer_params[NAME]
    learning_rate = optimizer_params[LEARNING_RATE]
    weight_decay = optimizer_params[WEIGHT_DECAY]

    if optimizer == "adam":
        return torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    elif optimizer == "adamw":
        return torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    elif optimizer == "rmsprop":
        momentum = optimizer_params[MOMENTUM]
        return torch.optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay, momentum=momentum)

In [14]:
LEARNING_RATE_SCHEDULER = "learning_rate_scheduler"
GAMMA = "gamma"
STEP_SIZE = "step_size"
FACTOR = "factor"


def get_learning_rate_scheduler(learning_rate_scheduler_params: dict, optimizer: torch.optim.Optimizer):
    learning_rate_scheduler = learning_rate_scheduler_params[NAME]
    if learning_rate_scheduler == "None":
        return None

    if learning_rate_scheduler == "step":
        step_size = learning_rate_scheduler_params[STEP_SIZE]
        gamma = learning_rate_scheduler_params[GAMMA]
        return torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
    elif learning_rate_scheduler == "exponential":
        gamma = learning_rate_scheduler_params[GAMMA]
        return torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma)
    elif learning_rate_scheduler == "constant":
        factor = learning_rate_scheduler_params[FACTOR]
        return torch.optim.lr_scheduler.ConstantLR(optimizer, factor=factor)

In [15]:
run_id = 0
SEED = 42


def train_efficient_net():
    global run_id
    run_id += 1

    L.seed_everything(SEED)

    wandb.init(name=f"efficientnet-{run_id}")

    wandb_logger = WandbLogger(log_model=True)

    # TODO: A lot of this could become a library

    config = wandb.config
    efficientnet_model = get_pretrained_efficientnet_model(config[EFFICIENTNET_MODEL])

    run_type = config[TUNE_TYPE]
    model = get_asl_efficientnet_model(run_type, efficientnet_model, dropout=config[DROPOUT], unfreeze_features=2)

    optimizer_params = config[OPTIMIZER]
    optimizer = get_optimizer(optimizer_params, model)

    learning_rate_scheduler_params = config[LEARNING_RATE_SCHEDULER]
    scheduler = get_learning_rate_scheduler(learning_rate_scheduler_params, optimizer)

    asl_model = ASLModel(model=model, criterion=nn.CrossEntropyLoss(), optimizer=optimizer, lr_scheduler=scheduler)

    train(
        model=asl_model,
        datamodule=datamodule,
        logger=wandb_logger,
        seed=SEED
    )

In [16]:
sweep_config = {
    "method": "random",
    "metric": {
        "name": f"{ASLModel.VALID_ACCURACY}",
        "goal": "maximize"
    },
    "early_terminate": {
        "type": "hyperband",
        "min_iter": 5
    },
    "parameters": {
        TUNE_TYPE: {
            "values": ["repurpose"] #, "finetune"]
        },
        EFFICIENTNET_MODEL: {
            "values": ["b0", "b1"] #, "b2", "b3"]
        },
        DROPOUT: {
            "min": 0.1,
            "max": 0.5
        },
        OPTIMIZER: {
            "parameters": {
                NAME: {
                    "values": ["adam", "adamw", "rmsprop"]
                },
                LEARNING_RATE: {
                    "min": 1e-5,
                    "max": 1e-2,
                    "distribution": "log_uniform_values"
                },
                WEIGHT_DECAY: {
                    "min": 0,
                    "max": 1e-3,
                },
                MOMENTUM: {
                    "min": 0.8,
                    "max": 0.99
                }
            }
        },
        LEARNING_RATE_SCHEDULER: {
            "parameters": {
                NAME: {
                    "values": ["None", "step", "exponential", "constant"]
                },
                STEP_SIZE: {
                    "min": 1,
                    "max": 10
                },
                GAMMA: {
                    "min": 0.1,
                    "max": 0.9
                },
                FACTOR: {
                    "min": 0.1,
                    "max": 0.5,
                }
            }
        }
    }
}

In [ ]:
sweep(sweep_config, 10, train_efficient_net)

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find ./dspro2/efficientnet.ipynb.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: yualbs7m
Sweep URL: https://wandb.ai/dspro2-silent-speech/silent-speech/sweeps/yualbs7m


wandb: Agent Starting Run: v0dxzle1 with config:
wandb: 	dropout: 0.4080004129219553
wandb: 	efficientnet_model: b0
wandb: 	learning_rate_scheduler: {'factor': 0.3813333361356862, 'gamma': 0.7275501907434817, 'name': 'constant', 'step_size': 6}
wandb: 	optimizer: {'learning_rate': 1.1579640831711996e-05, 'momentum': 0.9782010003214552, 'name': 'adamw', 'weight_decay': 3.534708758912542e-05}
wandb: 	tune_type: repurpose
Seed set to 42
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find ./dspro2/efficientnet.ipynb.
wandb: Currently logged in as: v8-luky (dspro2-silent-speech) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A16') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/opt/conda/lib/python3.12/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                     | Params | Mode 
--------------------------------------------------------------------
0 | model          | ASLEfficientNetRepurpose | 4.0 M  | train
1 | criteri

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Metric valid_accuracy improved. New best score: 0.049
Metric train_accuracy improved. New best score: 0.714


Validation: |          | 0/? [00:00<?, ?it/s]

Metric valid_accuracy improved by 0.037 >= min_delta = 0.0. New best score: 0.086


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

## Evaluation